# Angles in Low Barrier Hydrogen Bonds (LBHB)

### PDB file, additional table

| **Code** | **Description** |
| --- | --- |
| **fa_atr**                | Lennard-Jones attractive between atoms in different residues |
| **fa_rep**                | Lennard-Jones repulsive between atoms in different residues |
| **fa_sol**                | Lazaridis-Karplus solvation energy |
| **fa_intra_sol_xover4**   | Intra-residue Lazaridis-Karplus solvation energy |
| **lk_ball_wtd**           | Asymmetric solvation energy |
| **fa_intra_rep**          | Lennard-Jones repulsive between atoms in the same residue |
| **fa_elec**               | Coulombic electrostatic potential with a distance-dependent dielectric |   
| **pro_close**             | Proline ring closure energy and energy of psi angle of preceding residue |
| <font color='red'>**hbond_sr_bb**</font>          | <span style="color:red">Backbone-backbone hbonds close in primary sequence</span> |
| <span style="color:green">**hbond_lr_bb**</span>            | <span style="color:green">Backbone-backbone hbonds distant in primary sequence</span> |
| <span style="color:red">**hbond_bb_sc** </span>          | <span style="color:red">Sidechain-backbone hydrogen bond energy</span> |
| <span style="color:green">**hbond_sc**</span>              | <span style="color:green">Sidechain-sidechain hydrogen bond energy</span> |
| **dslf_fa13**             | Disulfide geometry potential |
| **rama_prepro**           | Ramachandran preferences (with separate lookup tables for pre-proline positions and other positions) |
| **omega**                 | Omega dihedral in the backbone. A Harmonic constraint on planarity with standard deviation of ~6 deg. |
| **p_aa_pp**               | Probability of amino acid, given torsion values for phi and psi |
| **fa_dun**                | Internal energy of sidechain rotamers as derived from Dunbrack's statistics |
| **yhh_planarity**         | A special torsional potential to keep the tyrosine hydroxyl in the plane of the aromatic ring |
| **ref**                   | Reference energy for each amino acid. Balances internal energy of amino acid terms.  Plays role in design. |
| **METHOD_WEIGHTS**        | Not an energy term itself, but the parameters for each amino acid used by the ref energy term. |







## Angles in any HB

In this project we are interest in two angles from acceptor and donor atomand their adjaccent atoms: $\hat{\epsilon}$ (epsilon) and $\hat{\theta}$ (theta). 

<img src="http://pubs.sciepub.com/ajme/3/2/3/image/fig1.png" width="80%"/>


 


## Side Chain Hydrogen bonds

In [8]:
import math
import pandas as pd
import sys
import numpy as np
from Bio.PDB import *

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.set_option('display.width',None)
pd.set_option('display.max_colwidth',None)

In [9]:
# Folders and files for processing 
srt_i =0#srt_i=int(sys.argv[1])
stp_i = 999999 #stp_i=int(sys.argv[2])
data_dir = "../data"
with open("../data/pdb_list.txt", encoding="utf-8") as f:
    read_data = f.read()
    pdb_list = read_data.split('\n')

print("number of pdbs = %s"%len(pdb_list))

if stp_i < len(pdb_list):
    pdb_list = pdb_list[srt_i:stp_i]
else:
    pdb_list = pdb_list[srt_i:]
    stp_i=len(pdb_list)
print("number of pdbs = %s"%len(pdb_list))
print(pdb_list[:10])

MAX_LBHB_DIST = 6.5
METAL_RES_CODES= [ "FE" ,"FE2","FES","FEO"
                 ,"CU" ,"CU1","CUA"
                 ,"MG" ,"ZN" ,"MN"
                 ,"MO" ,"MOO","MOS"
                 ,"NI" ,"3CO","CO"]

number of pdbs = 22268
number of pdbs = 22268
['121p_A', '13pk_A', '13pk_B', '13pk_C', '13pk_D', '1a0c_A', '1a0c_B', '1a0c_C', '1a0c_D', '1a0d_A']


In [10]:
# Extract residues, atoms and positions from  PDB files
def split_ATOM(raw):
    return(raw.str[6:11].str.replace(' ', ''),  
           raw.str[11:16].str.replace(' ', ''),
           raw.str[16].str.replace(' ', ''),
           raw.str[17:20].str.replace(' ', ''), 
           raw.str[21].str.replace(' ', ''),
           raw.str[22:26].str.replace(' ', ''),
           raw.str[27].str.replace(' ', ''),
           raw.str[30:37].str.replace(' ', ''),
           raw.str[38:45].str.replace(' ', ''),
           raw.str[46:53].str.replace(' ', ''),
           raw.str[54:59].str.replace(' ', ''),
           raw.str[60:65].str.replace(' ', ''),
           raw.str[72:75].str.replace(' ', ''),
           raw.str[76:78].str.replace(' ', ''),
           raw.str[79:].str.replace(' ', ''))

def get_ATOM_DF(ATOM, pdb=None):
    atom_data = split_ATOM(ATOM['raw'])
    ATOM = pd.DataFrame({
         'serial':atom_data[0],
         'atom_name':atom_data[1],
         'altLoc':atom_data[2],
         'resName':atom_data[3],
         'chainID':atom_data[4],
         'seqNum':atom_data[5],
         'iCode':atom_data[6],
         'x':atom_data[7],
         'y':atom_data[8],
         'z':atom_data[9],
         'occupancy':atom_data[10],
         'tempFactor':atom_data[11],
         'segID':atom_data[12],
         'element':atom_data[13],
         'charge':atom_data[14] })
    # change coordinates to float to make them usable
    ATOM['x']=ATOM['x'].astype(float)
    ATOM['y']=ATOM['y'].astype(float)
    ATOM['z']=ATOM['z'].astype(float)
    return(ATOM)

def get_struc_atom_coords(fullpath2pdbfile):
    f = open(fullpath2pdbfile, "r")
    pdb_lines = f.readlines()
    pdb_file_df = pd.DataFrame(pdb_lines, columns = ['raw'])
    pdb_file_df['raw'] = pdb_file_df['raw'].str[:80]
    pdb_file_df['key'] = pdb_file_df['raw'].str[:6].str.replace(' ', '')
    ## only keep first model, mostly for NMR models
    atom_coords = get_ATOM_DF(pdb_file_df[pdb_file_df['key'] == 'ATOM'])
    if len(pdb_file_df[pdb_file_df['key'] == 'HETATM'])>0:
        hetatm_coords = get_ATOM_DF(pdb_file_df[pdb_file_df['key'] == 'HETATM'])
        atom_coords = pd.concat([atom_coords, hetatm_coords], ignore_index=True)
    #atom_coords.reset_index(drop=True, inplace=True)
    return(atom_coords)


In [11]:
# Extract pKa atom positions
POTENTIAL_IONIZE_RES_ATOMS = [
    "ARG_NH1", "ARG_NH2", #"ARG_NE",
    "ASP_OD1", "ASP_OD2",
    "GLU_OE1", "GLU_OE2",
    "TYR_OH",
    "HIS_NE2", "HIS_ND1",
    "LYS_NZ", 'LYS_N', 'LYS_O'
    "CYS_SG"  ]

pka_file_cols = ["atom_name", "resName", "seqNum", "predicted_pKa",  "reference_pKa", 
                 "pKa_shift_desolvation_self-energy", "pKa_shift_background_interactions", 
                 "pKa_shift_charge-charge_interactions", "generalized_Born_radius", 
                 "solvent_exposure_parameter"]

def get_resi_pKas(pka_file):
    pka_data = pd.read_csv(pka_file, sep='\s+', names=pka_file_cols, skiprows=1)
    pka_data = pka_data.loc[~pka_data['atom_name'].isin(["N", "C"])]
    return(pka_data)

def get_pka_coords_data(this_pka_data, this_atom_coords, this_struc_id):
     for this_df in [this_pka_data, this_atom_coords]:
          this_df["atom_name"]=this_df["atom_name"].astype(str).str[:]
          this_df["resName"]=this_df["resName"].astype(str)
          this_df["seqNum"]=this_df["seqNum"].astype(int)

     del this_pka_data['atom_name']
     this_pka_coords_data = pd.merge(this_pka_data, this_atom_coords, 
                                     left_on=["resName", "seqNum"], 
                                     right_on=["resName", "seqNum"] )
     
     ## only keep atoms that could form a sidechain LBHB
     this_pka_coords_data['res_atom'] = (
          this_pka_coords_data['resName']+"_"+this_pka_coords_data['atom_name'])
     this_pka_coords_data = this_pka_coords_data.loc[
          this_pka_coords_data['res_atom'].isin(POTENTIAL_IONIZE_RES_ATOMS)]
     this_pka_coords_data.reset_index(drop=True, inplace=True)
     return(this_pka_coords_data)


In [13]:
# Get pair of residues, atom positions participating in side-chain Hydrogen bond
def get_LBHB_pairs(this_data_dir, this_struc_id):
    this_pka_file = "%s/%s_bluues.pka"%(this_data_dir, this_struc_id)
    this_pka_data = get_resi_pKas(this_pka_file)

    this_struc_file = "%s/%s_Rlx.pdb"%(this_data_dir, this_struc_id)
    this_atom_coords = get_struc_atom_coords(this_struc_file)
    
    this_pka_coords_data= get_pka_coords_data(this_pka_data, this_atom_coords, this_struc_id)
    print(this_pka_coords_data.shape)

    for col in ["atom_name", "resName", "seqNum", "predicted_pKa", "x", "y", "z"]:
        this_pka_coords_data.rename(columns={col:"%s2"%col}, inplace=True)

    ionizable_atom_pair_data = []
    ## get distances between all ionizable atoms
    for index, row in this_pka_coords_data[:-1].iterrows():
        unpaired_atoms = this_pka_coords_data[["atom_name2", "resName2", "seqNum2", 
                                               "predicted_pKa2", "x2", "y2", "z2"]].iloc[index+1:].copy()
        unpaired_atoms = unpaired_atoms.loc[unpaired_atoms["seqNum2"]!=row["seqNum2"]]
        for col in ["atom_name", "resName", "seqNum", "predicted_pKa", "x", "y", "z"]:
            unpaired_atoms["%s1"%col]=row["%s2"%col]
        ionizable_atom_pair_data.append(unpaired_atoms)
    ionizable_atom_pairs=pd.concat(ionizable_atom_pair_data)
    sq_difx = (ionizable_atom_pairs['x1']-ionizable_atom_pairs['x2'])**2
    sq_dify = (ionizable_atom_pairs['y1']-ionizable_atom_pairs['y2'])**2
    sq_difz = (ionizable_atom_pairs['z1']-ionizable_atom_pairs['z2'])**2
    ionizable_atom_pairs['distance']= sq_difx + sq_dify + sq_difz
    ionizable_atom_pairs['distance']=ionizable_atom_pairs['distance'].apply(math.sqrt)

    ## remove duplicate 
    ionizable_atom_pairs['bond_id'] = (ionizable_atom_pairs["resName1"].astype(str) + 
                                       ionizable_atom_pairs["seqNum1"].astype(str) + "_" +
                                       ionizable_atom_pairs["resName2"].astype(str) + 
                                       ionizable_atom_pairs["seqNum2"].astype(str))
    
    ionizable_atom_pairs.loc[ionizable_atom_pairs["seqNum1"]>ionizable_atom_pairs["seqNum2"],'bond_id'] = (
        ionizable_atom_pairs["resName2"].astype(str) + 
        ionizable_atom_pairs["seqNum2"].astype(str) + "_" +
        ionizable_atom_pairs["resName1"].astype(str) + 
        ionizable_atom_pairs["seqNum1"].astype(str))
    
    ionizable_atom_pairs.sort_values("distance", ascending=True, inplace=True)
    ionizable_atom_pairs.drop_duplicates(subset=['bond_id'], keep='first', inplace=True, ignore_index=False)
    ionizable_atom_pairs['pKa_diff']=ionizable_atom_pairs['predicted_pKa2']-ionizable_atom_pairs['predicted_pKa1']
    ionizable_atom_pairs['pKa_diff']=ionizable_atom_pairs['pKa_diff'].apply(abs)
    LBHB_data = ionizable_atom_pairs.loc[ionizable_atom_pairs['distance']<=MAX_LBHB_DIST].copy().reset_index(drop=True)
    
    for coord_axis in ['x', 'y', 'z']:
        LBHB_data[coord_axis]=LBHB_data["%s1"%(coord_axis)]+LBHB_data["%s2"%(coord_axis)]
        LBHB_data[coord_axis]=LBHB_data[coord_axis]/2

    LBHB_data['close_metal_resName']=""
    LBHB_data['close_metal_seqNum']=0
    LBHB_data['close_metal_distance']=10000

    metals = this_atom_coords.loc[this_atom_coords["resName"].isin(METAL_RES_CODES)]
    if len(metals)>0:
        for index, row in LBHB_data.iterrows():
            cur_metals = metals.copy()
            cur_metals['distance']=((cur_metals['x'].astype(float)-row['x'])**2 + 
                                    (cur_metals['y'].astype(float)-row['y'])**2 + 
                                    ((cur_metals['z'].astype(float)-row['z'])**2))
            cur_metals['distance']=cur_metals['distance'].apply(math.sqrt)
            min_dist = cur_metals['distance'].min()
            LBHB_data.loc[index, 'close_metal_distance'] = min_dist
            LBHB_data.loc[index, 'close_metal_resName']= cur_metals.loc[
                cur_metals['distance']==min_dist, 'resName'].tolist()[0]
            LBHB_data.loc[index, 'close_metal_seqNum']= cur_metals.loc[
                cur_metals['distance']==min_dist, 'seqNum'].tolist()[0]
    return(LBHB_data)

def get_LBHB_pairs_for_struc_list(data_directory, struc_list):
    all_LBHB_data = []
    for struc in struc_list:
        try:
          struc_LBHB_data = get_LBHB_pairs("%s/%s/%s/%s"%(data_directory, struc[0], struc[1], struc), struc)
          struc_LBHB_data['struc']=struc
          all_LBHB_data.append(struc_LBHB_data)
        except:
            print("failed to add LBHB for %s"%(struc))
      
    all_LBHB_pairs = pd.concat(all_LBHB_data, ignore_index=True)
    return(all_LBHB_pairs)




In [14]:
# example
path2pdb = '../data/7/a/7adh_A/7adh_A_Rlx.pdb'
adh7_pdb = get_struc_atom_coords(path2pdb) # ../data/7/a/7adh_A/7adh_A_Rlx.pdb

path2pka = '../data/7/a/7adh_A/7adh_A_bluues.pka'
adh7_pka = get_resi_pKas(path2pka)

adh7_pka_pdb = get_pka_coords_data(adh7_pka, adh7_pdb, this_struc_id ='7adh')

this_data_dir = '../data/7/a/7adh_A'
lbhb = get_LBHB_pairs(this_data_dir, this_struc_id ='7adh_A')
lbhb.to_csv("../results/LBHBs_adh7.csv")
len(lbhb)
lbhb[:2]

(178, 23)


,atom_name2,resName2,seqNum2,predicted_pKa2,x2,y2,z2,atom_name1,resName1,seqNum1,predicted_pKa1,x1,y1,z1,distance,bond_id,pKa_diff,x,y,z,close_metal_resName,close_metal_seqNum,close_metal_distance
0,NZ,LYS,19,11.571,-2.44,-6.83,9.90,OE1,GLU,16,3.756,-4.63,-6.15,10.99,2.539016,GLU16_LYS19,7.815,-3.535,-6.490,10.445,ZN,375,23.966259
1,OE1,GLU,239,4.115,25.08,16.27,43.27,NH2,ARG,218,13.581,24.62,16.42,45.90,2.674135,ARG218_GLU239,9.466,24.850,16.345,44.585,ZN,375,30.200602


### Angles for side chain Hydrogen Bonds

In [31]:
#________________________________________________________________________________________________________________
# Function to extract atoms adjacent-accpetor/donor-hydrogen for single LYS, ARG, HIS, ASP, GLU, TYR, CYS
def alys(residue):
  ce = [atom for atom in residue.get_atoms() if atom.get_name() == "CE"]
  nz = [atom for atom in residue.get_atoms() if atom.get_name() == "NZ"]
  lysa = [ce, nz]
  return lysa

def aarg(residue):
  cz = [atom for atom in residue.get_atoms() if atom.get_name() == "CZ"]
  nh1 = [atom for atom in residue.get_atoms() if atom.get_name() == "NH1"]
  nh2 = [atom for atom in residue.get_atoms() if atom.get_name() == "NH2"]
  arga1 = [cz, nh1]
  arga2 = [cz, nh2]
  return arga1 # improve to include arga2

def ahis(residue): 
  cg = [atom for atom in residue.get_atoms() if atom.get_name() == "CG"]
  nd1 = [atom for atom in residue.get_atoms() if atom.get_name() == "ND1"]
  cd2 = [atom for atom in residue.get_atoms() if atom.get_name() == "CD2"]
  ne2 = [atom for atom in residue.get_atoms() if atom.get_name() == "NE2"]
  hisa1 = [cg, nd1]
  hisa2 = [cd2, ne2] 
  return hisa1 # improve to include hisa2

def aasp(residue):
  cg = [atom for atom in residue.get_atoms() if atom.get_name() == "CG"]
  od1 = [atom for atom in residue.get_atoms() if atom.get_name() == "OD1"]
  od2 = [atom for atom in residue.get_atoms() if atom.get_name() == "OD2"]
  aspa1 = [cg, od1] #correct it
  aspa2 = [cg, od2] #correct it
  return aspa1 # improve to include aspa2]

def aglu(residue):
  cd = [atom for atom in residue.get_atoms() if atom.get_name() == "CD"]
  oe1 = [atom for atom in residue.get_atoms() if atom.get_name() == "OE1"]
  oe2 = [atom for atom in residue.get_atoms() if atom.get_name() == "OE2"]
  glua1 = [cd, oe1] #correct it
  glua2 = [cd, oe2] #correct it
  return glua1

def atyr(residue):
  cz = [atom for atom in residue.get_atoms() if atom.get_name() == "CZ"]
  oh = [atom for atom in residue.get_atoms() if atom.get_name() == "OH"]
  tyra = [cz, oh]
  return tyra

def acys(residue):
  cb = [atom for atom in residue.get_atoms() if atom.get_name() == "CB"]
  sg = [atom for atom in residue.get_atoms() if atom.get_name() == "SG"]
  cysa = [cb, sg]
  return cysa

#________________________________________________________________________________________________________________
# Function to extract residues from a protein structure
def prot_residues(structure, selector):
  residues = []
  for model in structure.get_models():
    for chain in model.get_chains():
      for residue in chain.get_residues():
        for i in range(len(selector)):
          if residue.get_id()[1] == selector[i]:
            residues.append(residue)
  return residues


#________________________________________________________________________________________________________________
# Function to extract the atoms for any HDBH residues
def res_atoms(residue):
  rname = residue.get_resname()
  res_atoms = alys(residue) if rname=='LYS' else aarg(residue) if rname=='ARG' else ahis(residue) if rname=='HIS' else aasp(residue) if rname=='ASP' else aglu(residue) if rname=='GLU' else atyr(residue) if rname=='TYR' else acys(residue)
  return res_atoms

#________________________________________________________________________________________________________________
# Function to extract the six atoms that coudl participate in the LBHB angle calculation
def lbhb_pair_atoms(pdb_file, lbhb_data):
  lbhb_residues1 =  lbhb_data["seqNum1"]
  lbhb_residues2 =  lbhb_data["seqNum2"]

  struct_id = pdb_file[-10:-14]
  p = PDBParser()
  s = p.get_structure(struct_id, pdb_file)

  charged_aa1 = prot_residues(s, lbhb_residues1)
  charged_aa2 = prot_residues(s, lbhb_residues2)

  four_atoms = []
  for i in range(len(charged_aa1)):
    res1 = res_atoms(charged_aa1[i])
    res2 = res_atoms(charged_aa2[i])
    lbhb_atoms = [res1[0], res1[1], res2[0], res2[1]]
    four_atoms.append(lbhb_atoms)
  return four_atoms

#________________________________________________________________________________________________________________
# Function to calculate the five main angles of LBHB
def single_angle(atom1, atom2, atom3):
  v1 = atom1[0].get_coord() - atom2[0].get_coord()
  v2 = atom3[0].get_coord() - atom2[0].get_coord()
  angle = np.degrees(np.arccos(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))))
  if angle < 90:
    angle += 90
  return round(angle, 3)

def sidechain_angles(fouratoms):
  epsilon = single_angle(fouratoms[0], fouratoms[1], fouratoms[3])
  theta = single_angle(fouratoms[2], fouratoms[3], fouratoms[1])
  angles = [epsilon, theta]
  return angles


#________________________________________________________________________________________________________________
# Function to calculate five_angles for all residues in a protein 

def protein_sidechain_angles(data_dir, struc, table_lbhb):
  path2pdb = "%s/%s/%s/%s/%s_Rlx.pdb"%(data_dir, struc[0], struc[1], struc, struc)
  four_atoms = lbhb_pair_atoms(path2pdb, table_lbhb)
  lbhb_sch_angles = table_lbhb.copy()
  lbhb_sch_angles[["epsilon", "theta", "structure", 'group']] = "", "", "", ""
  i = 0
  for atoms in four_atoms:
      try: 
          angles = sidechain_angles(atoms)
          lbhb_sch_angles['epsilon'][i] = angles[0]
          lbhb_sch_angles['theta'][i] = angles[1]
          lbhb_sch_angles['structure'][i] = struc
          lbhb_sch_angles['group'][i] = struc[:1]

      except:
          print('ERROR ' + str(i))
      i += 1
  return lbhb_sch_angles

#________________________________________________________________________________________________________________
# Function to calculate five_angles for all proteins in a directory 
def lbhb_mainangles(data_directory, struc_list):
    all_LBHB_pairs = []
    all_sch_angles = []
    for struc in struc_list:
        path2pdb = "%s/%s/%s/%s"%(data_directory, struc[0], struc[1], struc)
        try:
          protein_angles = protein_sidechain_angles(data_directory, struc, get_LBHB_pairs(path2pdb, struc))
          all_sch_angles.append(protein_angles)
        except:
          print("failed to add LBHB for %s"%(struc))

    all_LBHB_pairs = pd.concat(all_sch_angles, ignore_index=True)
    return(all_LBHB_pairs)


In [32]:
# Example
directory = '../data'
structure = '7adh_A'
lbhb_5angles = protein_sidechain_angles(directory, structure, lbhb)
lbhb_5angles.head(3)

/var/folders/v3/nq35dxlx2bxg58jt43fv2h4dcfpwcn/T/ipykernel_2578/4176612494.py:124: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lbhb_sch_angles['epsilon'][i] = angles[0]
/var/folders/v3/nq35dxlx2bxg58jt43fv2h4dcfpwcn/T/ipykernel_2578/4176612494.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lbhb_sch_angles['theta'][i] = angles[1]
/var/folders/v3/nq35dxlx2bxg58jt43fv2h4dcfpwcn/T/ipykernel_2578/4176612494.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ret

,atom_name2,resName2,seqNum2,predicted_pKa2,x2,y2,z2,atom_name1,resName1,seqNum1,predicted_pKa1,x1,y1,z1,distance,bond_id,pKa_diff,x,y,z,close_metal_resName,close_metal_seqNum,close_metal_distance,epsilon,theta,structure,group
0,NZ,LYS,19,11.571,-2.44,-6.83,9.90,OE1,GLU,16,3.756,-4.63,-6.15,10.99,2.539016,GLU16_LYS19,7.815,-3.535,-6.490,10.445,ZN,375,23.966259,171.805,143.855,7adh_A,7
1,OE1,GLU,239,4.115,25.08,16.27,43.27,NH2,ARG,218,13.581,24.62,16.42,45.90,2.674135,ARG218_GLU239,9.466,24.850,16.345,44.585,ZN,375,30.200602,132.425,177.46,7adh_A,7
2,N,LYS,19,11.571,2.76,-3.81,13.11,N,LYS,18,11.051,1.39,-3.51,15.42,2.702406,LYS18_LYS19,0.520,2.075,-3.660,14.265,ZN,375,18.415935,155.815,112.895,7adh_A,7


In [ ]:
data_directory = '../data'
struc_list = pdb_list[:len(pdb_list)-2] # 22251
allprots2 = lbhb_mainangles(data_directory, struc_list)
allprots2.to_csv("../results/LBHBs_adh7.csv")
print(len(allprots2))

In [30]:
import pandas as pd
df_angles = pd.read_csv("../results/LBHBs_angles.csv")
print(len(df_angles))

1690896


In [33]:
df_angles.head(2)

,Unnamed: 0,atom_name2,resName2,seqNum2,predicted_pKa2,x2,y2,z2,atom_name1,resName1,seqNum1,predicted_pKa1,x1,y1,z1,distance,bond_id,pKa_diff,x,y,z,close_metal_resName,close_metal_seqNum,close_metal_distance,epsilon,theta,structure,group
0,0,OE1,GLU,49,3.855,0.51,26.78,-6.09,OH,TYR,4,12.732,-0.19,25.15,-4.15,2.628783,TYR4_GLU49,8.877,0.160,25.965,-5.12,MG,168,27.256242,2.199724,1.480360,121p_A,1.0
1,1,OH,TYR,137,13.252,6.61,12.64,17.04,ND1,HIS,94,6.069,6.54,11.37,19.38,2.663344,HIS94_TYR137,7.183,6.575,12.005,18.21,MG,168,24.021173,2.050010,0.299128,121p_A,1.0


In [ ]:
import math
import plotly.express as px

fig = px.scatter_3d(df_angles, x='pKa_diff', y='theta', z='epsilon',
                    color='group', width=1200, height=720, 
                    labels={
                        "pKa_diff": "pKa Differences",
                        "epsilon": "Angle N2-N1-C1",
                        "theta": "Angle N1-N2-C2" } )
fig.update_traces(marker=dict(size=0.2))
fig.show()

In [46]:
path2pdb = '../data/7/a/7adh_A/7adh_A_Rlx.pdb'
adh7_pdb = get_struc_atom_coords(path2pdb) # ../data/7/a/7adh_A/7adh_A_Rlx.pdb

struct_id = path2pdb[-10:-14]
p = PDBParser()
s = p.get_structure(struct_id, path2pdb)
type(s)

Bio.PDB.Structure.Structure

In [ ]:
def prot_residues(structure, selector):
  residues = []
  for model in structure.get_models():
    for chain in model.get_chains():
      for residue in chain.get_residues():
        for i in range(len(selector)):
          if residue.get_id()[1] == selector[i]:
            residues.append(residue)
  return residues